In [1]:
import pandas as pd
import numpy as np

## Clean up building data

Start with the raw 2019 data from the City of Seattle.

Calculate extra fields:
- OSE Building Type: mapping between the City's Building Type and the building types used by OSE. See `city_building_types.csv` for mapping. These are generic categories like "Nonresidential" and "Multifamily".
- OSE Property Use fields: this is a mapping of EPA property use types to the types used by OSE. See `building_activity_types.csv` for mapping. These are more detailed categories like "Hospital" or "Restaurant".
- Property Use Type GFA for Policy fields: the GFA for Policy is the same as the GFA unless it is one of the use types "Data Center" and "Parking" that aren't subject to the policy (then it's 0).
- Total GFA for Policy: sum of the Property Use Type GFA for Policy fields
- PropertyUseType Percent GFA fields: percent of a building's total GFA that is from the given property use type. Zero if the use type is not covered by the policy

In [2]:
raw_building_data = pd.read_csv('../../data/input_data/seattle_large_building_data_2019.csv')

In [3]:
raw_building_data.columns

Index(['OSEBuildingID', 'TaxParcelIdentificationNumber', 'DataYear',
       'BuildingType', 'BuildingName', 'CouncilDistrictCode', 'Neighborhood',
       'YearBuilt', 'Address', 'City', 'State', 'ZipCode', 'Latitude',
       'Longitude', 'PrimaryPropertyType', 'ListOfAllPropertyUseTypes',
       'EPAPropertyType', 'TotalGHGEmissions', 'ENERGYSTARScore',
       'YearsENERGYSTARCertified', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
       'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SourceEUI(kBtu/sf)',
       'SourceEUIWN(kBtu/sf)', 'GHGEmissions(MetricTonsCO2e)',
       'GHGEmissionsIntensity', 'GHGEmissionsIntensity(kgCO2e/ft2)',
       'NaturalGas(kBtu)', 'NaturalGas(therms)', 'Electricity(kBtu)',
       'Electricity(kWh)', 'SteamUse(kBtu)', 'OtherFuelUse(kBtu)',
       'PropertyGFATotal', 'PropertyGFABuilding(s)', 'PropertyGFAParking',
       'NumberofBuildings', 'NumberofFloors', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
      

In [4]:
cleaned_building_data = raw_building_data[[
    'OSEBuildingID', 
    'TaxParcelIdentificationNumber', 
    'DataYear',
    'BuildingType', 
    'BuildingName', 
    'NaturalGas(kBtu)', 
    'Electricity(kBtu)', 
    'SteamUse(kBtu)', 
    'PropertyGFATotal', 
    'PropertyGFABuilding(s)', 
    'PropertyGFAParking', 
    'LargestPropertyUseType',
    'LargestPropertyUseTypeGFA', 
    'SecondLargestPropertyUseType',
    'SecondLargestPropertyUseTypeGFA', 
    'ThirdLargestPropertyUseType',
    'ThirdLargestPropertyUseTypeGFA'
]]

In [5]:
# Map OSE Building Type

city_building_types_mapping = pd.read_csv('../../data_new/input_data/city_building_types.csv')
building_types = dict(zip(list(city_building_types_mapping['BuildingType (City classification)']), list(city_building_types_mapping['Type (Legislative classification)'])))

In [6]:
cleaned_building_data['OSE Building Type'] = cleaned_building_data.apply(lambda building: building_types[building['BuildingType']], axis=1)

/var/folders/dv/sn1_d0xn0vncjbncdpkkg_pc0000gn/T/ipykernel_6458/356869480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_building_data['OSE Building Type'] = cleaned_building_data.apply(lambda building: building_types[building['BuildingType']], axis=1)


In [7]:
cleaned_building_data[['BuildingType', 'OSE Building Type']].head()

,BuildingType,OSE Building Type
0,NonResidential,NonResidential
1,NonResidential,NonResidential
2,NonResidential,NonResidential
3,NonResidential,NonResidential
4,NonResidential,NonResidential


In [8]:
# Map use types

In [9]:
ose_use_types = pd.read_csv('../../data_new/input_data/building_activity_types.csv')
ose_use_types.columns
ose_use_types_mapping = dict(zip(ose_use_types['EPA Building Type'], ose_use_types['OSE Building Type']))

In [10]:
cleaned_building_data['LargestPropertyUseType OSE'] = cleaned_building_data.apply(lambda building: ose_use_types_mapping[building['LargestPropertyUseType']], axis=1)
cleaned_building_data['SecondLargestPropertyUseType OSE'] = cleaned_building_data.apply(lambda building: ose_use_types_mapping[building['SecondLargestPropertyUseType']], axis=1)
cleaned_building_data['ThirdLargestPropertyUseType OSE'] = cleaned_building_data.apply(lambda building: ose_use_types_mapping[building['ThirdLargestPropertyUseType']], axis=1)

/var/folders/dv/sn1_d0xn0vncjbncdpkkg_pc0000gn/T/ipykernel_6458/4016502074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_building_data['LargestPropertyUseType OSE'] = cleaned_building_data.apply(lambda building: ose_use_types_mapping[building['LargestPropertyUseType']], axis=1)
/var/folders/dv/sn1_d0xn0vncjbncdpkkg_pc0000gn/T/ipykernel_6458/4016502074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_building_data['SecondLargestPropertyUseType OSE'] = cleaned_building_data.apply(lamb

In [11]:
cleaned_building_data['LargestPropertyUseType OSE'].unique()

array(['Hotel', 'Fire/Police Station', 'Entertainment/Public Assembly',
       'Multifamily Housing', 'Services', 'Recreation', 'Other',
       'K-12 School', 'College/University', 'Office',
       'Self-Storage Facility', 'Retail Store', 'Senior Living Community',
       'Supermarket/Grocery Store', 'Hospital',
       'Residence Hall/Dormitory', 'Non-Refrigerated Warehouse', nan,
       'Worship Facility', 'Laboratory', 'Restaurant',
       'Refrigerated Warehouse'], dtype=object)

In [12]:
cleaned_building_data[cleaned_building_data['LargestPropertyUseType OSE'].isnull()]['LargestPropertyUseType']

158         Parking
233     Data Center
239         Parking
247             NaN
283         Parking
           ...     
3524            NaN
3526            NaN
3527            NaN
3529            NaN
3532            NaN
Name: LargestPropertyUseType, Length: 99, dtype: object

In [13]:
# the buildings with parking/data center/etc are fine--those uses aren't covered by the policy so NaN is the right mapping
# but what's with these buildings with no EPA use type listed?
cleaned_building_data.iloc[3524]

OSEBuildingID                                                50529
TaxParcelIdentificationNumber                           1625049001
DataYear                                                      2019
BuildingType                                        NonResidential
BuildingName                        UW- New Life Sciences Building
NaturalGas(kBtu)                                                 0
Electricity(kBtu)                                                0
SteamUse(kBtu)                                                   0
PropertyGFATotal                                            210416
PropertyGFABuilding(s)                                      210416
PropertyGFAParking                                               0
LargestPropertyUseType                                         NaN
LargestPropertyUseTypeGFA                                      NaN
SecondLargestPropertyUseType                                   NaN
SecondLargestPropertyUseTypeGFA                               

In [14]:
# some buildings don't have a largest EPA use type at all
# we can't do anything about these buildings, so we'll drop them

cleaned_building_data = cleaned_building_data[~cleaned_building_data['LargestPropertyUseType'].isnull()]

In [15]:
cleaned_building_data[cleaned_building_data['LargestPropertyUseType'].isnull()]

,OSEBuildingID,TaxParcelIdentificationNumber,DataYear,BuildingType,BuildingName,NaturalGas(kBtu),Electricity(kBtu),SteamUse(kBtu),PropertyGFATotal,PropertyGFABuilding(s),...,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,OSE Building Type,LargestPropertyUseType OSE,SecondLargestPropertyUseType OSE,ThirdLargestPropertyUseType OSE


In [20]:
# Property Use Type GFA for Policy

cleaned_building_data['LargestPropertyUseType OSE GFA for Policy'] = cleaned_building_data.apply(lambda building: 0 if building['LargestPropertyUseType OSE'] in ('Data Center', 'Parking') else building['LargestPropertyUseTypeGFA'], axis=1)
cleaned_building_data['SecondLargestPropertyUseType OSE GFA for Policy'] = cleaned_building_data.apply(lambda building: 0 if building['SecondLargestPropertyUseType OSE'] in ('Data Center', 'Parking') else building['SecondLargestPropertyUseTypeGFA'], axis=1)
cleaned_building_data['ThirdLargestPropertyUseType OSE GFA for Policy'] = cleaned_building_data.apply(lambda building: 0 if building['ThirdLargestPropertyUseType OSE'] in ('Data Center', 'Parking') else building['ThirdLargestPropertyUseTypeGFA'], axis=1)

In [21]:
cleaned_building_data[['LargestPropertyUseType OSE GFA for Policy', 'SecondLargestPropertyUseType OSE GFA for Policy', 'ThirdLargestPropertyUseType OSE GFA for Policy']].head()

,LargestPropertyUseType OSE GFA for Policy,SecondLargestPropertyUseType OSE GFA for Policy,ThirdLargestPropertyUseType OSE GFA for Policy
0,88434.0,NaN,NaN
1,83880.0,15064.0,4622.0
2,756493.0,138635.0,0.0
3,61320.0,NaN,NaN
4,123445.0,68009.0,0.0


In [22]:
# replace NaN with 0
cleaned_building_data[['LargestPropertyUseType OSE GFA for Policy', 'SecondLargestPropertyUseType OSE GFA for Policy', 'ThirdLargestPropertyUseType OSE GFA for Policy']] = cleaned_building_data[['LargestPropertyUseType OSE GFA for Policy', 'SecondLargestPropertyUseType OSE GFA for Policy', 'ThirdLargestPropertyUseType OSE GFA for Policy']].fillna(0)

In [23]:
cleaned_building_data['Total GFA for Policy'] = cleaned_building_data.apply(lambda building: building['LargestPropertyUseType OSE GFA for Policy'] + building['SecondLargestPropertyUseType OSE GFA for Policy'] + building['ThirdLargestPropertyUseType OSE GFA for Policy'], axis=1)

In [24]:
# Calculate percent for each use type
cleaned_building_data['LargestPropertyUseType Percent GFA'] = cleaned_building_data.apply(lambda building: building['LargestPropertyUseType OSE GFA for Policy'] / building['Total GFA for Policy'], axis=1)
cleaned_building_data['SecondLargestPropertyUseType Percent GFA'] = cleaned_building_data.apply(lambda building: building['SecondLargestPropertyUseType OSE GFA for Policy'] / building['Total GFA for Policy'], axis=1)
cleaned_building_data['ThirdLargestPropertyUseType Percent GFA'] = cleaned_building_data.apply(lambda building: building['ThirdLargestPropertyUseType OSE GFA for Policy'] / building['Total GFA for Policy'], axis=1)

In [3]:
# Add size classification

def classify_size(sq_ft):
    """
    Use letter classifications for building size instead of dealing with size ranges (>220k, 90-220k, etc.)
    """
    if sq_ft > 220000:
        return 'A'
    elif sq_ft > 90000:
        return 'B'
    elif sq_ft > 50000:
        return 'C'
    elif sq_ft > 30000:
        return 'D'
    elif sq_ft > 20000:
        return 'E'
    else:
        return 'F'
        
cleaned_building_data['sq_ft_classification'] = cleaned_building_data['Total GFA for Policy'].apply(lambda building: classify_size(building))

In [4]:
cleaned_building_data.to_csv('cleaned_building_data_with_policy_gfa.csv')

## Rechecking the data

When we ran this data through the model, our numbers were way off. We need to figure out why that is. Here's some exploring to see where the problem is occuring.

In [2]:
raw_2019_data = pd.read_csv('seattle_large_building_data_2019.csv')
processed_data = pd.read_csv('cleaned_building_data_with_policy_gfa.csv')

In [3]:
rmi_processed_data = pd.read_csv('../rmi_building_analysis_with_new_col_names.csv')

In [5]:
# Let's look to see if there are missing buildings

# buildings not in RMI data but in the raw data
len(set(raw_2019_data['OSEBuildingID']) - set(rmi_processed_data['OSEBuildingID']))

1

In [6]:
set(raw_2019_data['OSEBuildingID']) - set(rmi_processed_data['OSEBuildingID'])

{755}

In [8]:
raw_2019_data.columns

Index(['OSEBuildingID', 'TaxParcelIdentificationNumber', 'DataYear',
       'BuildingType', 'BuildingName', 'CouncilDistrictCode', 'Neighborhood',
       'YearBuilt', 'Address', 'City', 'State', 'ZipCode', 'Latitude',
       'Longitude', 'PrimaryPropertyType', 'ListOfAllPropertyUseTypes',
       'EPAPropertyType', 'TotalGHGEmissions', 'ENERGYSTARScore',
       'YearsENERGYSTARCertified', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
       'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SourceEUI(kBtu/sf)',
       'SourceEUIWN(kBtu/sf)', 'GHGEmissions(MetricTonsCO2e)',
       'GHGEmissionsIntensity', 'GHGEmissionsIntensity(kgCO2e/ft2)',
       'NaturalGas(kBtu)', 'NaturalGas(therms)', 'Electricity(kBtu)',
       'Electricity(kWh)', 'SteamUse(kBtu)', 'OtherFuelUse(kBtu)',
       'PropertyGFATotal', 'PropertyGFABuilding(s)', 'PropertyGFAParking',
       'NumberofBuildings', 'NumberofFloors', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
      

In [10]:
print(raw_2019_data[raw_2019_data['OSEBuildingID'] == 755][['BuildingName', 'TotalGHGEmissions', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
       'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType',
       'ThirdLargestPropertyUseTypeGFA']])

                BuildingName  TotalGHGEmissions LargestPropertyUseType  \
546  MEDICAL DENTAL BUILDING           310839.6         Medical Office   

     LargestPropertyUseTypeGFA SecondLargestPropertyUseType  \
546                   315026.0                      Parking   

     SecondLargestPropertyUseTypeGFA ThirdLargestPropertyUseType  \
546                          28870.0                Retail Store   

     ThirdLargestPropertyUseTypeGFA  
546                         16587.0  


OK, so the Medical Dental building is huge (~360K sq ft) and produces a lot of carbon (300K tons)

It's possible this is throwing us off. It may have been removed as an outlier from the RMI dataset.

Let's also look at what's in the RMI but not the public raw dataset.

In [12]:
len(set(rmi_processed_data['OSEBuildingID']) - set(raw_2019_data['OSEBuildingID']))

44

In [13]:
in_rmi_but_not_public_city_data = list(set(rmi_processed_data['OSEBuildingID']) - set(raw_2019_data['OSEBuildingID']))

In [15]:
rmi_processed_data[rmi_processed_data['OSEBuildingID'].isin(in_rmi_but_not_public_city_data)][['LargestPropertyUseType', 'SecondLargestPropertyUseType', 'ThirdLargestPropertyUseType']].head()

,LargestPropertyUseType,SecondLargestPropertyUseType,ThirdLargestPropertyUseType
419,Other,NaN,NaN
747,Multifamily Housing,NaN,NaN
802,Multifamily Housing,Parking,Restaurant
901,Multifamily Housing,NaN,NaN
917,Multifamily Housing,NaN,NaN


## Create matching public/RMI datasets

Create a copy of the RMI data and the public data (crunched above) that have the exact same buildings (only buildings found in both datasets).